In [5]:
from datasets import load_dataset

In [6]:
ls dataset

caption.txt         Flickr8K_Text/      folder/
Flicker8k_Dataset/  Flickr8k.token.txt  my_test_image/


In [7]:
cat dataset/Flickr8k.token.txt

Singkapan batuan sedimen klastik dengan bidang perlapisan yang tidak tegas, batulumpur karbonatan, masif, retak-retak, sebagian hancur dan mulai lapuk
Singkapan batuan sedimen klastik dengan bidang perlapisan yang tidak tegas, masif, retak-retak, sebagian hancur sehingga mulai lapuk dan batulumpur karbonatan
Singkapan batuan sedimen klastik dan batulumpur karbonatan 
batulumpur karbonatan dan Singkapan batuan sedimen klastik
Singkapan batuan sedimen klastik dengan bidang perlapisan yang tidak tegas dan batulumpur karbonatan
Pecahan koral
Pecahan koral
Pecahan koral
Pecahan koral
Pecahan koral
Singkapan Batulumpur karbonatan
Singkapan batuan sedimen klastik Batulumpur karbonatan
batulumpur karbonatan dengan bidang perlapisan yang tidak tegas, masif, retak-retak, mulai lapuk dan sebagian hancur 
Singkapan batuan sedimen klastik batulumpur karbonatan dengan bidang perlapisan yang tidak tegas, retak-retak, masif, mulai lapuk dan sebagian hancur
Singkapan batuan sedimen klastik dengan bidan

In [8]:
dataset = load_dataset("text", data_files="dataset/Flickr8k.token.txt", split="train")

Using custom data configuration default-2966e04187faf8ed
Found cached dataset text (/home/dhiya/.cache/huggingface/datasets/text/default-2966e04187faf8ed/0.0.0/21a506d1b2b34316b1e82d0bd79066905d846e5d7e619823c0dd338d6f1fa6ad)


In [5]:
dataset["text"]

['Singkapan batuan sedimen klastik dengan bidang perlapisan yang tidak tegas, batulumpur karbonatan, masif, retak-retak, sebagian hancur dan mulai lapuk',
 'Singkapan batuan sedimen klastik dengan bidang perlapisan yang tidak tegas, masif, retak-retak, sebagian hancur sehingga mulai lapuk dan batulumpur karbonatan',
 'Singkapan batuan sedimen klastik dan batulumpur karbonatan ',
 'batulumpur karbonatan dan Singkapan batuan sedimen klastik',
 'Singkapan batuan sedimen klastik dengan bidang perlapisan yang tidak tegas dan batulumpur karbonatan',
 'Pecahan koral',
 'Pecahan koral',
 'Pecahan koral',
 'Pecahan koral',
 'Pecahan koral',
 'Singkapan Batulumpur karbonatan',
 'Singkapan batuan sedimen klastik Batulumpur karbonatan',
 'batulumpur karbonatan dengan bidang perlapisan yang tidak tegas, masif, retak-retak, mulai lapuk dan sebagian hancur ',
 'Singkapan batuan sedimen klastik batulumpur karbonatan dengan bidang perlapisan yang tidak tegas, retak-retak, masif, mulai lapuk dan sebagia

In [6]:
def get_training_corpus():
    for i in range(0, len(dataset), 1000):
        yield dataset[i : i + 1000]["text"]

In [7]:
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))

In [8]:
tokenizer.normalizer = normalizers.BertNormalizer(lowercase=True)

In [9]:
tokenizer.normalizer = normalizers.Sequence(
    [normalizers.NFD(), normalizers.Lowercase(), normalizers.StripAccents()]
)

In [10]:
print(tokenizer.normalizer.normalize_str("Héllò hôw are ü?"))

hello how are u?


In [11]:
tokenizer.pre_tokenizer = pre_tokenizers.BertPreTokenizer()

In [12]:
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.WordPieceTrainer(vocab_size=25000, special_tokens=special_tokens)

In [13]:
tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)

In [14]:
encoding = tokenizer.encode("batuan andesit where.")
print(encoding.tokens)

['batuan', 'andesit', 'w', '##h', '##er', '##e', '.']


In [15]:
cls_token_id = tokenizer.token_to_id("[CLS]")
sep_token_id = tokenizer.token_to_id("[SEP]")
print(cls_token_id, sep_token_id)

2 3


In [16]:
tokenizer.post_processor = processors.TemplateProcessing(
    single=f"[CLS]:0 $A:0 [SEP]:0",
    pair=f"[CLS]:0 $A:0 [SEP]:0 $B:1 [SEP]:1",
    special_tokens=[("[CLS]", cls_token_id), ("[SEP]", sep_token_id)],
)

In [17]:
encoding = tokenizer.encode("Andesit berwarna coklat dan abu-abu tua yang bertekstur hornblende dengan tekstur warna abu.")
print(encoding.tokens)

['[CLS]', 'andesit', 'berwarna', 'coklat', 'dan', 'abu', '-', 'abu', 'tua', 'yang', 'bertekstur', 'hornblende', 'dengan', 'tekstur', 'warna', 'abu', '.', '[SEP]']


In [18]:
encoding = tokenizer.encode("andesit merah Let's test this tokenizer...", "batuan on a pair of sentences.")
print(encoding.tokens)
print(encoding.type_ids)

['[CLS]', 'andesit', 'merah', 'let', '[UNK]', 's', 't', '##es', '##t', 't', '##h', '##is', '[UNK]', '.', '.', '.', '[SEP]', 'batuan', 'o', '##n', 'a', 'pa', '##ir', 'o', '##f', 'se', '##n', '##t', '##en', '##c', '##es', '.', '[SEP]']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [19]:
tokenizer.decoder = decoders.WordPiece(prefix="##")

In [20]:
tokenizer.decode(encoding.ids)

'andesit merah let s test this... batuan on a pair of sentences.'

In [21]:
tokenizer.save("tokenizer.json")

In [22]:
new_tokenizer = Tokenizer.from_file("tokenizer.json")

In [23]:
new_tokenizer

## Text Model

In [12]:
datasets = load_dataset("text", data_files="dataset/Flickr8k.token.txt")

Using custom data configuration default-2966e04187faf8ed
Found cached dataset text (/home/dhiya/.cache/huggingface/datasets/text/default-2966e04187faf8ed/0.0.0/21a506d1b2b34316b1e82d0bd79066905d846e5d7e619823c0dd338d6f1fa6ad)


  0%|          | 0/1 [00:00<?, ?it/s]

In [13]:
datasets

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 660
    })
})

In [10]:
from transformers import PreTrainedTokenizerFast
from tokenizers import Tokenizer

wrapped_tokenizer = PreTrainedTokenizerFast(
    tokenizer_file="model/tokenizer.json", # You can load from the tokenizer file, alternatively
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    mask_token="[MASK]",
)

In [1]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/242M [00:00<?, ?B/s]

In [30]:
batch_size = 16

args = Seq2SeqTrainingArguments(
    output_dir="./trainer/output",
    logging_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=50,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [15]:
args

Seq2SeqTrainingArguments(output_dir='./trainer/output', overwrite_output_dir=False, do_train=False, do_eval=True, do_predict=False, evaluation_strategy=<IntervalStrategy.EPOCH: 'epoch'>, prediction_loss_only=False, per_device_train_batch_size=16, per_device_eval_batch_size=16, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, eval_delay=0, learning_rate=2e-05, weight_decay=0.01, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=1, max_steps=-1, lr_scheduler_type=<SchedulerType.LINEAR: 'linear'>, warmup_ratio=0.0, warmup_steps=0, log_level='passive', log_level_replica='passive', log_on_each_node=True, logging_dir='./trainer/output/runs/Nov08_00-39-16_mayihd', logging_strategy=<IntervalStrategy.STEPS: 'steps'>, logging_first_step=False, logging_steps=500, logging_nan_inf_filter=True, save_strategy=<IntervalStrategy.STEPS: 'steps'>, save_steps=500, save_total_limit=3, save_on_ea

In [14]:
def preprocess_function(examples):
    inputs = wrapped_tokenizer(
        examples["text"],
        padding="max_length",
        max_length=20,
        truncation=True,
        return_token_type_ids=True,
        return_attention_mask=True,
        add_special_tokens=True,
    )
    
    inputs["labels"] = inputs["input_ids"]
    
    return inputs

tokenized_datasets = datasets.map(preprocess_function)

  0%|          | 0/660 [00:00<?, ?ex/s]

In [16]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 660
    })
})

In [17]:
data_collator = DataCollatorForSeq2Seq(wrapped_tokenizer, model=model)

In [31]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    data_collator=data_collator,
    tokenizer=wrapped_tokenizer,
)

In [32]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: text, token_type_ids. If text, token_type_ids are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 660
  Num Epochs = 50
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2100


Step,Training Loss
42,0.164700
84,0.133500
126,0.110400
168,0.091700
210,0.071000
252,0.059500
294,0.060700
336,0.068600
378,0.058300
420,0.045400


Saving model checkpoint to ./trainer/output/checkpoint-500
Configuration saved in ./trainer/output/checkpoint-500/config.json
Model weights saved in ./trainer/output/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./trainer/output/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./trainer/output/checkpoint-500/special_tokens_map.json
Saving model checkpoint to ./trainer/output/checkpoint-1000
Configuration saved in ./trainer/output/checkpoint-1000/config.json
Model weights saved in ./trainer/output/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./trainer/output/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./trainer/output/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to ./trainer/output/checkpoint-1500
Configuration saved in ./trainer/output/checkpoint-1500/config.json
Model weights saved in ./trainer/output/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in ./trainer/output/checkpo

TrainOutput(global_step=2100, training_loss=0.03391053707826705, metrics={'train_runtime': 118.7479, 'train_samples_per_second': 277.9, 'train_steps_per_second': 17.685, 'total_flos': 174464040960000.0, 'train_loss': 0.03391053707826705, 'epoch': 50.0})

In [33]:
trainer.save_model("model/stone-seq2seq")

Saving model checkpoint to model/stone-seq2seq
Configuration saved in model/stone-seq2seq/config.json
Model weights saved in model/stone-seq2seq/pytorch_model.bin
tokenizer config file saved in model/stone-seq2seq/tokenizer_config.json
Special tokens file saved in model/stone-seq2seq/special_tokens_map.json
